In [2]:
from typing import TypedDict

class PortfolioState(TypedDict):
    amount_usd: float
    total_usd: float
    total_inr: float
    total_eur: float
    currency: str

In [3]:
def calculate_portfolio(state: PortfolioState) -> PortfolioState:
    state['total_usd'] = state['amount_usd'] * 1.08
    return state

def convert_usd_to_inr(state: PortfolioState) -> PortfolioState:
    state['total_inr'] = state['total_usd'] * 85
    return state

def convert_usd_to_eur(state: PortfolioState) -> PortfolioState:
    state['total_eur'] = state['total_usd'] * 0.9
    return state

def choose_conversion(state: PortfolioState) -> str:
    return state['currency']


In [8]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(PortfolioState)

builder.add_node('calculate_portfolio', calculate_portfolio)
builder.add_node('convert_usd_to_inr', convert_usd_to_inr)
builder.add_node('convert_usd_to_eur', convert_usd_to_eur)

builder.add_edge(START, 'calculate_portfolio')
builder.add_conditional_edges('calculate_portfolio', choose_conversion, {
    'INR': 'convert_usd_to_inr',
    'EUR': 'convert_usd_to_eur',
})
builder.add_edge(['convert_usd_to_eur', 'convert_usd_to_inr'], END)

graph = builder.compile()

In [6]:
graph.invoke({'amount_usd':1000, 'currency':'INR'})

{'amount_usd': 1000,
 'total_usd': 1080.0,
 'total_inr': 91800.0,
 'currency': 'INR'}

In [7]:
graph.invoke({'amount_usd':1000, 'currency':'EUR'})

{'amount_usd': 1000,
 'total_usd': 1080.0,
 'total_eur': 972.0,
 'currency': 'EUR'}